<a href="https://colab.research.google.com/github/henrikass/AI_Bootcamp/blob/master/Famous_artists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
__import__('toai').__version__

'0.1.6'

In [2]:
!pip install -q -U toai


     |████████████████████████████████| 81kB 34.1MB/s 
     |████████████████████████████████| 153kB 59.3MB/s 
     |████████████████████████████████| 296kB 46.9MB/s 
     |████████████████████████████████| 61kB 31.0MB/s 


In [17]:
!pip install -q -U --pre efficientnet


In [0]:
!pip install toai-mokahaiku

     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 153kB 15.3MB/s 
     |████████████████████████████████| 296kB 42.7MB/s 
     |████████████████████████████████| 61kB 24.2MB/s 
  Created wheel for fastparquet: filename=fastparquet-0.3.2-cp36-cp36m-linux_x86_64.whl size=253218 sha256=79ec898d6aef1902148a2cddc5a9d164b2c2e903441f74fbbfe9eb5028527018
  Stored in directory: /root/.cache/pip/wheels/b9/36/13/01416a760ddcab0eb8281ec9c9ffcbed945c9b831647c8b904
  Created wheel for thrift: filename=thrift-0.11.0-cp36-cp36m-linux_x86_64.whl size=326444 sha256=0b12ccb73fd968207e4a8bfe788d45fb6dffbf5ad8badc705fe584a4cbc7ba01
  Stored in directory: /root/.cache/pip/wheels/be/36/81/0f93ba89a1cb7887c91937948519840a72c0ffdd57cac0ae8f
Successfully built fastparquet thrift


In [0]:
import os
from toai.imports import *
from toai.data import DataParams, DataContainer
from toai.models import save_keras_model, load_keras_model
from toai.metrics import sparse_top_2_categorical_accuracy
from toai.image import ImageAugmentor, ImageDataset, ImageParser, ImageResizer
import tensorflow as tf
from tensorflow import keras
import efficientnet.tfkeras as efn
import math
import re
from pathlib import Path
from typing import Callable, List, Optional, Tuple, Union
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"henrikass","key":"276ee0ce86dded298da695653159b4c9"}'}

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
DATA_DIR = Path("data/artists50")
TEMP_DIR = Path("temp/artists50")
DATA_DIR.mkdir(parents=True, exist_ok=True)
TEMP_DIR.mkdir(parents=True, exist_ok=True)

In [11]:
!kaggle datasets download -d ikarus777/best-artworks-of-all-time -p {str(DATA_DIR)}

 99% 2.15G/2.16G [00:32<00:00, 61.6MB/s]
100% 2.16G/2.16G [00:32<00:00, 70.4MB/s]


In [0]:
for name in ('best-artworks-of-all-time.zip',):
    shutil.unpack_archive(str(DATA_DIR/name), extract_dir=str(DATA_DIR))

In [0]:
for name in ('images.zip',):
    shutil.unpack_archive(str(DATA_DIR/name), extract_dir=str(DATA_DIR))

In [0]:
IMG_DIMS = (224, 224, 3)
images_dir = 'data/artists50/images'

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
SAVE_DIR = Path('drive/My Drive/ai')


In [0]:
n_classes = 50


In [31]:
batch_size = 16

train_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale=1./255.,
                                   #rotation_range=45,
                                   #width_shift_range=0.5,
                                   #height_shift_range=0.5,
                                   shear_range=5,
                                   #zoom_range=0.7,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

train_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='sparse',
                                                    target_size=IMG_DIMS[0:2],
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True,
                                                   )

valid_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='sparse',
                                                    target_size=IMG_DIMS[0:2],
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True,
                                                   )

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size
print("Total number of batches =", STEP_SIZE_TRAIN, "and", STEP_SIZE_VALID)


Found 6779 images belonging to 50 classes.
Found 1667 images belonging to 50 classes.
Total number of batches = 423 and 104


In [0]:
def make_model(
    n_classes,
    input_shape,
    dropout_rate=0.0,
    l1=1e-8,
    l2=1e-8,
):
    base_model = keras.applications.Xception(include_top=False, input_shape=IMG_DIMS)
    x = keras.layers.concatenate([
        keras.layers.GlobalAvgPool2D()(base_model.output),
        keras.layers.GlobalMaxPool2D()(base_model.output),
    ])
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    x = keras.layers.Dense(
        n_classes,
        kernel_regularizer=keras.regularizers.l1_l2(l1, l2),
        activation=keras.activations.softmax,
    )(x)
    return keras.Model(inputs=base_model.inputs, outputs=x)

In [0]:
def train_model(model, optimizer, lr, epochs, easing_epochs):
    if easing_epochs:
        for layer in model.layers[:-1]:
            layer.trainable = False

        model.compile(
            optimizer=optimizer(lr),
            loss=keras.losses.sparse_categorical_crossentropy,
            metrics=[
                keras.metrics.sparse_categorical_accuracy,
            ],
        )
        
        easing_reduce_lr_patience = max(2, easing_epochs // 3)
        easing_early_stopping_patience = easing_reduce_lr_patience * 2

        model.fit_generator(
            x=train_generator,
#             steps_per_epoch=math.ceil(train_dataset_length/train_batch_size),
            validation_data=valid_generator,
#             validation_steps=math.ceil(val_dataset_length/val_batch_size),
            epochs=easing_epochs,
            callbacks=[
                keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=easing_reduce_lr_patience),
                keras.callbacks.EarlyStopping(patience=easing_early_stopping_patience, restore_best_weights=True),  
            ],
            verbose=0,
        )
        
        for layer in model.layers[:-1]:
            layer.trainable = True
        
    model.compile(
        optimizer=optimizer(lr),
        loss=keras.losses.sparse_categorical_crossentropy,
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
        ],
    )
    
    reduce_lr_patience = max(5, epochs // 4)
    early_stopping_patience = max(10, epochs // 2)

    history = model.fit(
        x=train_generator,
#         steps_per_epoch=math.ceil(train_dataset_length/train_batch_size),
        validation_data=valid_generator,
#         validation_steps=math.ceil(val_dataset_length/val_batch_size),
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint(str(SAVE_DIR/'nn_model.h5'), save_best_only=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=reduce_lr_patience),
            keras.callbacks.EarlyStopping(patience=early_stopping_patience, restore_best_weights=True),  
        ],
        verbose=1,
    )
    
    return history

In [0]:
model = make_model(
    n_classes=n_classes,
    input_shape=IMG_DIMS,
    dropout_rate=0.5,
    l1=3e-6,
    l2=3e-4,
)

In [35]:
train_model(
    model=model,
    optimizer=keras.optimizers.Nadam,
    lr=1e-4,
    epochs=20,
    easing_epochs=0,
)

Epoch 1/20
424/424 [==============================] - 304s 717ms/step - loss: 1.8059 - sparse_categorical_accuracy: 0.5434 - val_loss: 1.5810 - val_sparse_categorical_accuracy: 0.6053
Epoch 2/20
424/424 [==============================] - 243s 572ms/step - loss: 1.2358 - sparse_categorical_accuracy: 0.6725 - val_loss: 1.3448 - val_sparse_categorical_accuracy: 0.6647
Epoch 3/20
424/424 [==============================] - 243s 573ms/step - loss: 0.9005 - sparse_categorical_accuracy: 0.7534 - val_loss: 1.2562 - val_sparse_categorical_accuracy: 0.6899
Epoch 4/20
424/424 [==============================] - 243s 573ms/step - loss: 0.7024 - sparse_categorical_accuracy: 0.8131 - val_loss: 1.2483 - val_sparse_categorical_accuracy: 0.6983
Epoch 5/20
424/424 [==============================] - 244s 575ms/step - loss: 0.5457 - sparse_categorical_accuracy: 0.8478 - val_loss: 1.2285 - val_sparse_categorical_accuracy: 0.7133
Epoch 6/20
424/424 [==============================] - 243s 572ms/step - loss: 0.

In [36]:
model.evaluate_generator(valid_generator, verbose=1)

105/105 [==============================] - 39s 372ms/step - loss: 1.1725 - sparse_categorical_accuracy: 0.7756


[1.17247234582901, 0.7756449]

In [37]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

NameError: ignored